In [129]:
import numpy as np
import pandas as pd
import scipy.linalg as lin

In [348]:
def load_data(input):
    # 以后这样读数据
    # 一开始就变成array了，就不用一直担心是list还是array了，而且一直可以用numpy
    # regex, \s space, + at least once, use what to seperate the origin string
    data = pd.read_csv(input, sep='\s+', header = None)
    # easily process as numpy array  (NOT matrix)
    data = data.as_matrix()
    row, col = data.shape
    X = data[:, 0:col - 1]
    Y = data[:, col - 1: col]
    return X, Y

def ada_boost(X, Y, T):
    # 注意用的都是一维向量，初始化用np.zeros((T,))
    row, col = X.shape
    U = np.ones((row, 1))/row
    Xsort = np.sort(X, 0)
    thres = (np.r_[Xsort[0:1,:] - 0.1, Xsort[0:row-1, :]] + Xsort) / 2
    theta = np.zeros((T,))
    s = np.zeros((T,))
    index = np.zeros((T,)).astype(int)
    alpha = np.zeros((T,))
    err = np.zeros((T,))
    
    for i in range(T):
        # one gt for each iteration
        err[i], theta[i], s[i], index[i] = decision_stump(X, Y, thres, U)
        yhat = s[i] * np.sign(X[:, index[i]:index[i]+1]-theta[i])
        delta = np.sqrt((1-err[i])/err[i])
        # this is interesting, use boolean for index
        U[yhat == Y] /= delta
        U[yhat != Y] *= delta
        # question 14
        #if i == T-1:
        #    print('sum(U): ', np.sum(U))
        alpha[i] = np.log(delta)
        U /= np.sum(U)  # guarantee sum U = 1
    # question 16
    #print(np.min(err))
    return theta, index, s, alpha

def decision_stump(X, Y, thres, U):
    row, col = X.shape
    r, c = thres.shape
    besterr = 1; btheta = 0; bs = 0; index = 0
    for i in range(col):
        # 先要想清楚矩阵结构，i和theta的组合，所以矩阵是把xi复制theta长度遍，做i的循环
        # 矩阵之间加减法和布尔运算与.dot是完全不同的，第n项对第n项，比如这里是左边第一行全部 - 右边第一行(也就是1)
        # 要搞懂这个矩阵运算，注意下面这两种写法是等价的。利用矩阵运算简化（免得用for 循环）
        #Yhat1 = np.sign(np.tile(X[:, i:i+1], (1, r)).T - thres[:,i:i+1]).T
        r, c = thres.shape
        Yhat1 = np.sign(X[:, i:i+1] - thres[:, i:i+1].T) # 或.reshape(1,r)
        
        err1 = (Yhat1!=Y).T.dot(U)
        err2 = (-1*Yhat1!=Y).T.dot(U)
        s = 1 if np.min(err1) < np.min(err2) else -1
        if s == 1 and np.min(err1) < besterr:
            besterr = np.min(err1)
            bs = 1
            index = i
            btheta = thres[np.argmin(err1), i]  # find the index of the min in array err1
        if s == -1 and np.min(err2) < besterr:
            besterr = np.min(err2)
            bs = -1
            index = i
            btheta = thres[np.argmin(err2), i]
    return besterr, btheta, bs, index
    
# 预测函数
def predict(X, theta, index, s, alpha):
    row, col = X.shape
    num = len(theta)
    # 矩阵和矩阵，一一对应
    # s.reshape((1, num))是放在矩阵里，俩括号
    # X[:, index]并不是一列，而是很多列, 把index这个向量当做索引，结果是一个矩阵，第i列是X[index[i]]
    #ytemp = np.tile(s, (row, 1))*np.sign(X[:, index]-theta)
    # 构造矩阵的思路要明确，i和t的矩阵
    ytemp = s*np.sign(X[:, index]-theta)
    yhat = np.sign(ytemp.dot(alpha.reshape(num, 1)))
    return yhat

def main():
    TRAIN_DATA = 'adaboost_train.dat'
    TEST_DATA = 'adaboost_test.dat'
    T = 300
    
    X_train, Y_train = load_data(TRAIN_DATA)
    row, col = X_train.shape
    X_test, Y_test = load_data(TEST_DATA)
    theta, index, s, alpha = ada_boost(X_train, Y_train, T)
    yhat_train = predict(X_train, theta, index, s, alpha)
    yhat_test = predict(X_test, theta, index, s, alpha)
    print('Ein(g1)：', np.sum(yhat_train!=Y_train)/row)
    print('Eout(g1)：', np.sum(yhat_test!=Y_test)/len(Y_test))
    
if __name__ == '__main__':
    main()

Ein(g1)： 0.0
Eout(g1)： 0.132


In [352]:
def matK(X, X1, gamma):
    row, col = X.shape
    r, c = X1.shape
    K = np.zeros((row, r))
    for i in range(r):
        K[:,i] = np.exp(-gamma*np.sum((X - X1[i,:])**2, 1))
    return K

def main():
    FULL_DATA = 'lssvm_all.dat'
    X_full, Y_full = load_data(FULL_DATA)
    Xtrain = X_full[0:400, :]
    Ytrain = Y_full[0:400, :]
    Xtest = X_full[400: ,:]
    Ytest = Y_full[400: ,:]
    
    gamma = [32, 2, 0.125]
    lamb = [0.001, 1, 1000]
    Ein = np.zeros((len(gamma), len(lamb)))
    Eout = np.zeros((len(gamma), len(lamb)))
    for i in range(len(gamma)):
        K = matK(Xtrain, Xtrain, gamma[i])
        K2 = matK(Xtrain, Xtest, gamma[i])
        for j in range(len(lamb)):
            beta = lin.pinv(lamb[j]*np.eye(len(Ytrain))+K).dot(Ytrain)
            # predict
            # K和beta的有两种可能组合，两种互为转置矩阵，而且一种奇大，一种很小
            # K并不是对称的，一边是X, 一边是X1，而要对应Y，所以有讲究
            # 其实很简单，为什么一个很大，是因为和Y的方向不同，所以比较了m*m次，而应该只比较m次
            Y_in_predict = np.sign(K.T.dot(beta))
            #print(np.sum(np.sign(K.T.dot(beta)).T != np.sign(beta.T.dot(K))))
            Ein[i,j] = np.sum(Ytrain != Y_in_predict)/len(Ytrain)
            Y_out_predict = np.sign(K2.T.dot(beta))
            Eout[i,j] = np.sum(Ytest != Y_out_predict)/len(Ytest)
        
    print('最小的Ein: ', np.min(Ein))
    print('最小的Eout: ', np.min(Eout))

if __name__ == '__main__':
    main()

最小的Ein:  0.0
最小的Eout:  0.39
